<a href="https://colab.research.google.com/github/RyanYavari/MarineVisionFilters/blob/main/law_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Law's Filters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# computing the local energy over a window of the image
def compute_texture_energy(filtered_img, window_size):
    # creating window kernel initialized to 1.0
    kernel = np.ones((window_size, window_size), dtype=np.float32)
    # normalizing kernel
    kernel = kernel / (window_size ** 2)

    # Compute the local sum of the absolute values of the filtered image
    texture_energy = cv2.filter2D(np.abs(filtered_img), -1, kernel)
    return texture_energy

Law's function that takes information from greyscale image

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import cv2
import os

def generate_laws(input_img, input_img_path, laws_filters, window_size, texture_count):
    # converting to cv2 greyscale
    input_img_grey = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)

    # convolving each filter with each image
    filtered_images = [cv2.filter2D(input_img_grey, -1, kernel) for kernel in laws_filters]

    # computing texture energies
    texture_energies = [compute_texture_energy(img, window_size) for img in filtered_images]

    # creating feature vectors the length of the energies
    feature_vectors = np.zeros((input_img_grey.shape[0], input_img_grey.shape[1], len(texture_energies)))

    # populate the feature vectors
    for i, energy_map in enumerate(texture_energies):
        feature_vectors[:, :, i] = energy_map

    # reshaping for clustering
    num_pixels = feature_vectors.shape[0] * feature_vectors.shape[1]
    feature_vectors_2d = feature_vectors.reshape(num_pixels, -1) # shaping so that each row is a sample, and each column is a feature

    # normalizing feature vectors, so that each feature has a mean of 0
    # each feature will contribute equally to k means clustering
    scaler = StandardScaler()
    feature_vectors_normalized = scaler.fit_transform(feature_vectors_2d)

    # apply k-means clustering
    k = texture_count  # number of textures to detect
    kmeans = KMeans(n_clusters=k, random_state=18)
    kmeans.fit(feature_vectors_normalized)
    cluster_labels = kmeans.labels_

    # reshape the cluster labels back to the 2D image shape
    segmentation_map = cluster_labels.reshape(feature_vectors.shape[0], feature_vectors.shape[1])

    # creating color map for visualization
    plt.figure(figsize=(10, 10))
    plt.imshow(segmentation_map, cmap='nipy_spectral')

    # extract the base name and extension of the input image
    base_name = os.path.splitext(os.path.basename(input_img_path))[0]
    output_dir = '/content/drive/Shareddrives/visionFinal/Laws_Images'
    output_file = f"{output_dir}/{base_name}_segmented.png"

    # save the image
    plt.savefig(output_file)
    plt.close()

Color Law's function that takes information from each hsv channel **Takes a long time to run (at least 5 mins)**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import cv2
import os

def generate_laws_color(input_img, input_img_path, laws_filters, window_size, texture_count):
    # splitting input images into color channels hsv
    input_img_hsv = cv2.cvtColor(input_img, cv2.COLOR_BGR2HSV)
    # crerating list of channel images
    channels = cv2.split(input_img_hsv)

    # creating list to store texture energies for all channels
    all_texture_energies = []

    for channel in channels:
        # convolving each filter with each channel of the image
        filtered_images = [cv2.filter2D(channel, -1, kernel) for kernel in laws_filters]

        # computing texture energies
        texture_energies = [compute_texture_energy(img, window_size) for img in filtered_images]

        # adding texture energies of this channel to the list
        all_texture_energies.extend(texture_energies)

    height, width = channels[0].shape

    # creating feature vectors the length of the energies from all channels
    feature_vectors = np.zeros((height, width, len(texture_energies))) # these vectors are 3 times larger vs greyscale

    # populate the feature vectors
    for i, energy_map in enumerate(texture_energies):
        feature_vectors[:, :, i] = energy_map

    # reshaping for clustering
    num_pixels = feature_vectors.shape[0] * feature_vectors.shape[1]
    feature_vectors_2d = feature_vectors.reshape(num_pixels, -1) # shaping so that each row is a sample, and each column is a feature

    # normalizing feature vectors, so that each feature has a mean of 0
    # each feature will contribute equally to k means clustering
    scaler = StandardScaler()
    feature_vectors_normalized = scaler.fit_transform(feature_vectors_2d)

    # apply k-means clustering
    k = texture_count  # number of textures to detect
    kmeans = KMeans(n_clusters=k, random_state=18) # random state is a random seed number
    kmeans.fit(feature_vectors_normalized) #
    cluster_labels = kmeans.labels_

    # reshape the cluster labels back to the 2D image shape
    segmentation_map = cluster_labels.reshape(feature_vectors.shape[0], feature_vectors.shape[1])

    # creating color map for visualization
    plt.figure(figsize=(10, 10))
    plt.imshow(segmentation_map, cmap='nipy_spectral') # cmap defines the color scheme

    # extract the base name and extension of the input image
    base_name = os.path.splitext(os.path.basename(input_img_path))[0]
    output_dir = '/content/drive/Shareddrives/visionFinal/Laws_Images_Color'
    output_file = f"{output_dir}/{base_name}_color_segmented.png"

    # save the image
    plt.savefig(output_file)
    plt.close()

"Main" function

In [ ]:
import cv2
import glob
import os
import numpy as np

# define the 1D vectors
L5 = np.array([1, 4, 6, 4, 1]) #Level
E5 = np.array([-1, -2, 0, 2, 1]) #Edge
S5 = np.array([-1, 0, 2, 0, -1]) #Spot
R5 = np.array([1, -4, 6, -4, 1]) #Ripple
W5 = np.array([-1, 2, 0, -2, 1]) #Wave

# creating Laws masks
laws_filters = []
for vec1 in (L5, E5, S5, R5, W5):
  for vec2 in (L5, E5, S5, R5, W5):
    # adding outer product of the vectors to filters list
    laws_filters.append(np.outer(vec1, vec2))

input_folder = '/content/drive/Shareddrives/visionFinal/ARMSPLATES'

window_size = 15 # how large should the texture window be
texture_count = 10 # number of textures to search for in image

# loop over all images in the input folder
for image_path in glob.glob(os.path.join(input_folder, '*.ppm')):
    # reading the image
    image = cv2.imread(image_path)

    # call the generate_laws function
    generate_laws(image, image_path, laws_filters, window_size, texture_count)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
